# Main purpose
This jupyter notebook show you how to download the approved drug list by extracting the information from the cancer.gov website

import packages

In [2]:
from get_approved_drug import get_approved_drug

In [3]:
## 
url = "https://www.cancer.gov/about-cancer/treatment/drugs/leukemia#4"
# url = "https://www.cancer.gov/about-cancer/treatment/drugs/lung#4"
##


query = "acute myeloid leukemia"
# query = " small cell lung cancer"

approved_drugs_df = get_approved_drug(url, query)

In [5]:
approved_drugs_df.head()

,drug_name,resource_url
0,Arsenic Trioxide,https://www.cancer.gov/about-cancer/treatment/...
1,Azacitidine,https://www.cancer.gov/about-cancer/treatment/...
2,Cerubidine (Daunorubicin Hydrochloride),https://www.cancer.gov/about-cancer/treatment/...
3,Cyclophosphamide,https://www.cancer.gov/about-cancer/treatment/...
4,Cytarabine,https://www.cancer.gov/about-cancer/treatment/...


In [9]:
# Save the dataframe
approved_drugs_df.to_csv(f'{query.replace(" ", "_")}_approved_drugs.csv', index=False)